In [1]:
!pip install 'zarr<3'
!pip install timm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [18]:
import zarr
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import os

# USER INPUTS - Change these values
dataset = 'ORION-CRC' ## or Xenium
case = 'CRC34'
modality = "mif"  # or "he"
sample_number = 5  # or 'random' for random selection

# Set paths
base_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/{dataset}/zarr_data/{case}"
zarr_path = os.path.join(base_path, modality, "images.zarr")

# Open the zarr array
z = zarr.open(zarr_path, mode='r')

# Get available chunks
chunks = [d for d in os.listdir(zarr_path) if d.endswith('.0.0.0')]
available_samples = sorted([int(d.split('.')[0]) for d in chunks])
print(f"Available samples: {available_samples}")

# Determine chunk index
if sample_number == 'random':
    chunk_idx = random.choice(available_samples)
    print(f"Randomly selected sample: {chunk_idx}")
else:
    chunk_idx = sample_number

print(f"Converting chunk {chunk_idx} to PNG...")

# Read the data
data = z[chunk_idx]

# Create output directory
output_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/"
os.makedirs(output_dir, exist_ok=True)

if modality == "he":
    # Handle HE images (3-channel RGB)
    if data.dtype == np.float32 or data.dtype == np.float64:
        data = ((data - data.min()) / (data.max() - data.min()) * 255).astype(np.uint8)
    elif data.dtype != np.uint8:
        data = data.astype(np.uint8)
    
    if len(data.shape) == 3 and data.shape[2] == 3:
        img = Image.fromarray(data, mode='RGB')
    elif len(data.shape) == 2:
        img = Image.fromarray(data, mode='L')
    else:
        img = Image.fromarray(data[:, :, 0], mode='L')
    
    output_path = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}.png")
    img.save(output_path)
    print(f"Saved to {output_path}")

elif modality == "mif":
    # Handle MIF images (2-channel)
    print(f"MIF image shape: {data.shape}")
    
    # Extract channels
    mif_ch0 = data[:, :, 0]  # Nuclei channel
    mif_ch1 = data[:, :, 1]  # Second channel
    
    # Normalize each channel to 0-255
    def normalize_channel(channel):
        ch_min, ch_max = channel.min(), channel.max()
        if ch_max > ch_min:
            return ((channel - ch_min) / (ch_max - ch_min) * 255).astype(np.uint8)
        return np.zeros_like(channel, dtype=np.uint8)
    
    mif_ch0_norm = normalize_channel(mif_ch0)
    mif_ch1_norm = normalize_channel(mif_ch1)
    
    # Create RGB image: Channel 0 (blue), Channel 1 (green/cyan)
    # R: Channel 1, G: Channel 1, B: Channel 0
    rgb_image = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.uint8)
    rgb_image[:, :, 0] = mif_ch1_norm  # Red channel = Channel 1
    rgb_image[:, :, 1] = mif_ch1_norm  # Green channel = Channel 1 (makes it cyan)
    rgb_image[:, :, 2] = mif_ch0_norm  # Blue channel = Channel 0
    
    # Save combined RGB image
    img_combined = Image.fromarray(rgb_image, mode='RGB')
    output_path_combined = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_combined.png")
    img_combined.save(output_path_combined)
    print(f"Saved combined RGB image to {output_path_combined}")
    
    # Also save individual channel visualizations for reference
    # Channel 0 (hot colormap)
    plt.figure(figsize=(10, 10))
    plt.imshow(mif_ch0, cmap='hot')
    plt.axis('off')
    plt.title('MIF Channel 0 (Nuclei)', fontsize=14)
    output_path_ch0 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch0.png")
    plt.savefig(output_path_ch0, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()
    print(f"Saved Channel 0 to {output_path_ch0}")
    
    # Channel 1 (viridis colormap)
    plt.figure(figsize=(10, 10))
    plt.imshow(mif_ch1, cmap='viridis')
    plt.axis('off')
    plt.title('MIF Channel 1', fontsize=14)
    output_path_ch1 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch1.png")
    plt.savefig(output_path_ch1, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()
    print(f"Saved Channel 1 to {output_path_ch1}")
    
    # Save side-by-side comparison
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))
    
    axes[0].imshow(mif_ch0, cmap='Blues')
    axes[0].set_title('Channel 0 (Nuclei - Blue)', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(mif_ch1, cmap='Greens')
    axes[1].set_title('Channel 1 (Cyan)', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(rgb_image)
    axes[2].set_title('Combined (Ch0=Blue, Ch1=Cyan)', fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    output_path_comparison = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_comparison.png")
    plt.savefig(output_path_comparison, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"Saved comparison to {output_path_comparison}")

Available samples: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Converting chunk 5 to PNG...
MIF image shape: (512, 512, 2)
Saved combined RGB image to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_combined.png
Saved Channel 0 to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_ch0.png
Saved Channel 1 to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_ch1.png
Saved comparison to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_comparison.png


In [3]:
import zarr
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

# USER INPUTS - Change these values
dataset = 'Xenium'  ## or Xenium
case = 'ovarian' ## lung lungv1 lymph_node ovarian pancreasV1 prostate skin 
modalities = ["he", "mif"]  # Process both modalities

# Create main output directory for this case
main_output_dir = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/{case}"
os.makedirs(main_output_dir, exist_ok=True)

# Process each modality
for modality in modalities:
    print(f"\n{'='*60}")
    print(f"Processing modality: {modality.upper()}")
    print(f"{'='*60}")
    
    # Set paths
    base_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/{dataset}/zarr_data/{case}"
    zarr_path = os.path.join(base_path, modality, "images.zarr")
    
    # Check if zarr path exists
    if not os.path.exists(zarr_path):
        print(f"Warning: Zarr path not found for {modality}: {zarr_path}")
        print(f"Skipping {modality}...")
        continue
    
    # Open the zarr array
    try:
        z = zarr.open(zarr_path, mode='r')
    except Exception as e:
        print(f"Error opening zarr for {modality}: {e}")
        continue
    
    # Get available chunks
    chunks = [d for d in os.listdir(zarr_path) if d.endswith('.0.0.0')]
    available_samples = sorted([int(d.split('.')[0]) for d in chunks])
    print(f"Available samples: {available_samples}")
    print(f"Total samples to process: {len(available_samples)}")
    
    # Create output subdirectory for this modality
    output_dir = os.path.join(main_output_dir, modality)
    os.makedirs(output_dir, exist_ok=True)
    
    # Process all samples
    for idx, chunk_idx in enumerate(available_samples, 1):
        print(f"\nProcessing {modality.upper()} sample {idx}/{len(available_samples)}: chunk {chunk_idx}...")
        
        try:
            # Read the data
            data = z[chunk_idx]
            
            if modality == "he":
                # Handle HE images (3-channel RGB)
                if data.dtype == np.float32 or data.dtype == np.float64:
                    data = ((data - data.min()) / (data.max() - data.min()) * 255).astype(np.uint8)
                elif data.dtype != np.uint8:
                    data = data.astype(np.uint8)
                
                if len(data.shape) == 3 and data.shape[2] == 3:
                    img = Image.fromarray(data, mode='RGB')
                elif len(data.shape) == 2:
                    img = Image.fromarray(data, mode='L')
                else:
                    img = Image.fromarray(data[:, :, 0], mode='L')
                
                output_path = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}.png")
                img.save(output_path)
                print(f"Saved to {output_path}")
            
            elif modality == "mif":
                # Handle MIF images (2-channel)
                print(f"MIF image shape: {data.shape}")
                
                # Extract channels
                mif_ch0 = data[:, :, 0]  # Nuclei channel
                mif_ch1 = data[:, :, 1]  # Second channel
                
                # Normalize each channel to 0-255
                def normalize_channel(channel):
                    ch_min, ch_max = channel.min(), channel.max()
                    if ch_max > ch_min:
                        return ((channel - ch_min) / (ch_max - ch_min) * 255).astype(np.uint8)
                    return np.zeros_like(channel, dtype=np.uint8)
                
                mif_ch0_norm = normalize_channel(mif_ch0)
                mif_ch1_norm = normalize_channel(mif_ch1)
                
                # Create RGB image: Channel 0 (blue), Channel 1 (cyan)
                rgb_image = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.uint8)
                rgb_image[:, :, 0] = 0  # Red channel = Channel 1
                rgb_image[:, :, 1] = mif_ch1_norm  # Green channel = Channel 1 (makes it cyan)
                rgb_image[:, :, 2] = mif_ch0_norm  # Blue channel = Channel 0

                
                # Save combined RGB image
                img_combined = Image.fromarray(rgb_image, mode='RGB')
                output_path_combined = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_combined.png")
                img_combined.save(output_path_combined)
                print(f"Saved combined RGB image to {output_path_combined}")
                
                # Save individual channel visualizations
                plt.figure(figsize=(10, 10))
                plt.imshow(mif_ch0, cmap='hot')
                plt.axis('off')
                plt.title('MIF Channel 0 (Nuclei)', fontsize=14)
                output_path_ch0 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch0.png")
                plt.savefig(output_path_ch0, bbox_inches='tight', pad_inches=0, dpi=150)
                plt.close()
                print(f"Saved Channel 0 to {output_path_ch0}")
                
                plt.figure(figsize=(10, 10))
                plt.imshow(mif_ch1, cmap='viridis')
                plt.axis('off')
                plt.title('MIF Channel 1', fontsize=14)
                output_path_ch1 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch1.png")
                plt.savefig(output_path_ch1, bbox_inches='tight', pad_inches=0, dpi=150)
                plt.close()
                print(f"Saved Channel 1 to {output_path_ch1}")
                
                # Save side-by-side comparison
                fig, axes = plt.subplots(1, 3, figsize=(30, 10))
                
                axes[0].imshow(mif_ch0, cmap='Blues')
                axes[0].set_title('Channel 0 (Nuclei - Blue)', fontsize=14, fontweight='bold')
                axes[0].axis('off')
                
                axes[1].imshow(mif_ch1, cmap='Greens')
                axes[1].set_title('Channel 1 (Cyan)', fontsize=14, fontweight='bold')
                axes[1].axis('off')
                
                axes[2].imshow(rgb_image)
                axes[2].set_title('Combined (Ch0=Blue, Ch1=Cyan)', fontsize=14, fontweight='bold')
                axes[2].axis('off')
                
                plt.tight_layout()
                output_path_comparison = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_comparison.png")
                plt.savefig(output_path_comparison, bbox_inches='tight', dpi=150)
                plt.close()
                print(f"Saved comparison to {output_path_comparison}")
        
        except Exception as e:
            print(f"Error processing chunk {chunk_idx} for {modality}: {e}")
            continue
    
    print(f"\n✓ {modality.upper()} processing complete: {len(available_samples)} samples processed")

print(f"\n{'='*60}")
print(f"ALL PROCESSING COMPLETE!")
print(f"Output directory: {main_output_dir}")
print(f"{'='*60}")



Processing modality: HE
Available samples: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93]
Total samples to process: 94

Processing HE sample 1/94: chunk 0...
Saved to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/ovarian/he/ovarian-he_chunk_0.png

Processing HE sample 2/94: chunk 1...
Saved to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/ovarian/he/ovarian-he_chunk_1.png

Processing HE sample 3/94: chunk 2...
Saved to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/ovarian/he/ovarian-he_chunk_2.png

Processing HE sample 4/94: chunk 3...
Saved to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/ovarian/he/ovarian-he_

In [11]:
import zarr
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

# USER INPUTS - Change these values
zarr_path = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/pannuke/zarr_data/Fold3/pannuke_mosaic_47/images.zarr"  # Path to your zarr file
chunk_idx = 0  # Which tile/chunk to extract
modality = "he"  # "he" or "mif"
output_dir = "/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images"  # Where to save the PNG

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Open the zarr array
z = zarr.open(zarr_path, mode='r')

# Read the specific tile
data = z[chunk_idx]
print(f"Tile shape: {data.shape}, dtype: {data.dtype}")

if modality == "he":
    # Handle HE images (3-channel RGB)
    if data.dtype == np.float32 or data.dtype == np.float64:
        data = ((data - data.min()) / (data.max() - data.min()) * 255).astype(np.uint8)
    elif data.dtype != np.uint8:
        data = data.astype(np.uint8)
    
    if len(data.shape) == 3 and data.shape[2] == 3:
        img = Image.fromarray(data, mode='RGB')
    elif len(data.shape) == 2:
        img = Image.fromarray(data, mode='L')
    else:
        img = Image.fromarray(data[:, :, 0], mode='L')
    
    output_path = os.path.join(output_dir, f"pannuke_fold3_tile_{chunk_idx}.png")
    img.save(output_path)
    print(f"Saved HE image to {output_path}")

elif modality == "mif":
    # Handle MIF images (2-channel)
    mif_ch0 = data[:, :, 0]  # Nuclei channel
    mif_ch1 = data[:, :, 1]  # Second channel
    
    # Normalize each channel to 0-255
    def normalize_channel(channel):
        ch_min, ch_max = channel.min(), channel.max()
        if ch_max > ch_min:
            return ((channel - ch_min) / (ch_max - ch_min) * 255).astype(np.uint8)
        return np.zeros_like(channel, dtype=np.uint8)
    
    mif_ch0_norm = normalize_channel(mif_ch0)
    mif_ch1_norm = normalize_channel(mif_ch1)
    
    # Create RGB image: Ch0=Blue, Ch1=Cyan
    rgb_image = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.uint8)
    rgb_image[:, :, 0] = 0  # Red
    rgb_image[:, :, 1] = mif_ch1_norm  # Green
    rgb_image[:, :, 2] = mif_ch0_norm  # Blue
    
    # Save combined RGB
    img_combined = Image.fromarray(rgb_image, mode='RGB')
    output_path = os.path.join(output_dir, f"tile_{chunk_idx}_combined.png")
    img_combined.save(output_path)
    print(f"Saved MIF combined image to {output_path}")

print("Done!")

Tile shape: (512, 512, 3), dtype: uint8
Saved HE image to /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/pannuke_fold3_tile_0.png
Done!
